In [9]:
import torch
import numpy as np

from torch_geometric.data import Data
from torch.nn.utils.rnn import pad_sequence
from grid2op_env.grid_to_gym import Grid_Gym
from evaluation.restore_agent import restore_agent

In [4]:
env_config = {'act_on_single_substation': True,
            'convert_to_tuple': True,
            'env_name': 'rte_case14_realistic',
            'keep_actions': 
                            ['change_bus'],
            'keep_observations': ['rho',
                                'gen_p',
                                'load_p',
                                'p_or',
                                'p_ex',
                                'timestep_overflow',
                                'maintenance',
                                'topo_vect'],
            'log_reward': False,
            'medha_actions': True,
            'reward_scaling_factor': 3,
            'rho_threshold': 0.9,
            'run_until_threshold': True,
            'scale': True,
            'use_parametric': False,
            'conn_matrix': True
            }

rllib_env = Grid_Gym(env_config);
env = rllib_env.org_env

def vectorize_obs(obs, env, hazard_threshold = 0.9):
   """
   Vectorize the gym observation.

   :param OrderedDict obs: gym observation
   :param Grid2Op_environment env: grid2op environment. Used to fetch the 
      ids of different objects.
   :param float hazard_threshold
   """

   length = env.action_space.dim_topo # number of bus bars == number of nodes in the graph

   # rho is symmetric for both ends of the line [56,1]
   rho = torch.zeros(length)
   rho[env.action_space.line_or_pos_topo_vect] = torch.from_numpy(obs["rho"])
   rho[env.action_space.line_ex_pos_topo_vect] = torch.from_numpy(obs["rho"])

   # active power p [56,1]
   p = torch.zeros(length)
   p[env.action_space.gen_pos_topo_vect] = torch.from_numpy(obs["gen_p"]) # generator active production
   p[env.action_space.load_pos_topo_vect] = torch.from_numpy(obs["load_p"]) # load active consumption
   p[env.action_space.line_or_pos_topo_vect] = torch.from_numpy(obs["p_or"]) # origin active flow
   p[env.action_space.line_ex_pos_topo_vect] = torch.from_numpy(obs["p_ex"]) # Extremity active flow

   # overflow [56,1]
   over = torch.zeros(length)
   over[env.action_space.line_or_pos_topo_vect] = torch.from_numpy(obs["timestep_overflow"]).float()
   over[env.action_space.line_ex_pos_topo_vect] = torch.from_numpy(obs["timestep_overflow"]).float()

   # one-hot topo vector [56,3]
   topo_vect_one_hot = torch.zeros(length,3)
   topo_vect = obs["topo_vect"]
   topo_vect[topo_vect==-1] = 0 # change disconneted from -1 to 0
   topo_vect_one_hot = torch.nn.functional.one_hot(torch.from_numpy(topo_vect).to(torch.int64), num_classes=3)

   # powerline maintenance
   # maintenance = torch.zeros(length)
   # maintenance[env.action_space.line_or_pos_topo_vect] = torch.from_numpy(obs["maintenance"]).float()
   # maintenance[env.action_space.line_ex_pos_topo_vect] = torch.from_numpy(obs["maintenance"]).float()

   # manual feature thresholding 
   hazard = torch.zeros(length) # [56,1]
   hazard[env.action_space.line_or_pos_topo_vect] = (torch.from_numpy(obs["rho"]) > hazard_threshold).float()
   hazard[env.action_space.line_ex_pos_topo_vect] = (torch.from_numpy(obs["rho"]) > hazard_threshold).float()

   vectorized_obs = torch.stack([rho,p,over, hazard], dim = 1)
   vectorized_obs = torch.concat([vectorized_obs, topo_vect_one_hot], dim = -1)
   
   return vectorized_obs

SUBSTATION NUMBER: 0
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 1
Choosing 3 out of 6
Choosing 4 out of 6
Choosing 5 out of 6
Choosing 6 out of 6
SUBSTATION NUMBER: 2
Choosing 2 out of 4
Choosing 3 out of 4
Choosing 4 out of 4
SUBSTATION NUMBER: 3
Choosing 3 out of 6
Choosing 4 out of 6
Choosing 5 out of 6
Choosing 6 out of 6
SUBSTATION NUMBER: 4
Choosing 3 out of 5
Choosing 4 out of 5
Choosing 5 out of 5
SUBSTATION NUMBER: 5
Choosing 3 out of 6
Choosing 4 out of 6
Choosing 5 out of 6
Choosing 6 out of 6
SUBSTATION NUMBER: 6
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 7
Choosing 1 out of 2
Choosing 2 out of 2
SUBSTATION NUMBER: 8
Choosing 3 out of 5
Choosing 4 out of 5
Choosing 5 out of 5
SUBSTATION NUMBER: 9
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 10
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 11
Choosing 2 out of 3
Choosing 3 out of 3
SUBSTATION NUMBER: 12
Choosing 2 out of 4
Choosing 3 out of 4
Choosing 4 out of 4
SUBS

/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/lightsim2grid/_aux_add_trafo.py:33: UserWarning: There were some Nan in the pp_net.trafo["tap_neutral"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_neutral\"], they have been replaced by 0")
/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/lightsim2grid/_aux_add_trafo.py:41: UserWarning: There were some Nan in the pp_net.trafo["tap_step_percent"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_step_percent\"], they have been replaced by 0")
/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/lightsim2grid/_aux_add_trafo.py:46: UserWarning: There were some Nan in the pp_net.trafo["tap_pos"], they have been replaced by 0
  warnings.warn("There were some Nan in the pp_net.trafo[\"tap_pos\"], they have been replaced by 0")
/opt/miniconda3/envs/rlib_grid/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: B

In [5]:
obs = rllib_env.reset()


In [10]:
vectorize_obs(obs, env).shape

torch.Size([56, 7])

In [7]:
cm = obs["connectivity_matrix"]

In [14]:
edgelist_with_self  = np.argwhere(cm==1)
edge_list = np.array([pair for pair in edgelist_with_self if pair[0] != pair[1]])

edge_list = torch.from_numpy(edge_list)

In [15]:
edge_index = torch.tensor([[0, 1],
                           [1, 0],
                           [1, 2],
                           [2, 1]], dtype=torch.long)
edge_index.shape

torch.Size([4, 2])

In [16]:
data = Data(x= vectorize_obs(obs, env), edge_index=edge_list.t().contiguous())